<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/cpu_gpu_qpu_hybrid_computing_mkbahk_20251202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**이 예제는 다음과 같은 시나리오를 따릅니다:**

1. CPU: 무작위 센서 데이터를 생성합니다.

2. GPU: AI 모델이 이 데이터를 분석하여 '특징값(Feature)'을 추출합니다.

3. CPU: 추출된 특징값을 양자 회로의 '회전 각도(Angle)'로 변환합니다.

4. QPU: 해당 각도로 양자 상태를 조작하고 측정하여 확률적인 결정을 내립니다.

5. CPU: 최종 결과를 해석합니다.

In [3]:
!pip install torch numpy qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.8 MB/s eta 0:00:00


**코드 상세 설명 및 아키텍처 흐름**

이 코드는 **서로** 다른 연산 장치 간의 데이터 핸드오버(Data Handover) 과정을 보여줍니다.

Phase 1: CPU → GPU (데이터 전송)

    역할: CPU는 I/O 작업을 처리하고 numpy 배열을 생성합니다.

    핵심: tensor.to(device) 명령어를 통해 데이터를 시스템 RAM(Host)에서 GPU VRAM(Device)으로 복사합니다. 대규모 데이터 처리 시 이 대역폭(Bandwidth)이 병목이 될 수 있습니다.

Phase 2: GPU 처리 (AI 추론)

    역할: 병렬 연산에 최적화된 GPU가 행렬 곱셈을 수행하여 패턴을 분석합니다.

    결과: 복잡한 입력 데이터(벡터)를 하나의 스칼라 값(확률)으로 압축하여 반환합니다.

Phase 3: CPU → QPU (인코딩)

    역할: GPU의 결과값은 보통 실수(Float)입니다. 이를 양자 상태로 변환하기 위해 각도 인코딩(Angle Encoding) 방식을 사용했습니다.

    수식: AI 출력값 $x \in [0, 1]$을 양자 회전각 θ=x×π로 변환합니다.

Phase 4: QPU 처리 (양자 연산)

    역할: Ry 게이트를 통해 큐비트를 중첩 상태 α∣0⟩+β∣1⟩로 만듭니다. 여기서 AI가 예측한 확률이 높을수록 상태 ∣1⟩이 측정될 확률이 높아지도록 물리적으로 조작합니다.

    참고: 실제 하드웨어에서는 이 단계에서 양자 얽힘이나 간섭을 이용해 고전 컴퓨터가 풀기 어려운 최적화 문제를 풉니다. (위 예제는 시뮬레이터인 Aer를 사용했습니다.)

**실제 환경 구현 시 고려사항**

    지연 시간 (Latency): CPU, GPU, QPU 간의 데이터 이동 시간은 매우 큽니다. 따라서 실시간 처리가 필요한 경우, 데이터 이동을 최소화하거나 비동기(Asynchronous) 처리가 필수적입니다.

    QPU 접근: 위 코드는 로컬 시뮬레이터를 사용했지만, 실제 양자 컴퓨터(IBM Quantum, IonQ 등)를 사용하려면 클라우드 API를 호출해야 하며, 큐(Queue) 대기 시간이 발생합니다.

    에러 완화: 현재의 QPU(NISQ)는 노이즈가 많으므로, CPU에서 에러 완화(Error Mitigation) 후처리가 필요할 수 있습니다.


In [4]:
import torch
import torch.nn as nn
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import math

In [5]:
# --- [1] GPU 파트: AI 모델 정의 ---
class SimpleClassifier(nn.Module):
    def __init__(self):
        super(SimpleClassifier, self).__init__()
        # 간단한 신경망: 입력 5 -> 은닉 10 -> 출력 1 (0~1 사이의 확률값)
        self.layer1 = nn.Linear(5, 10)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()
    ###def

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out
    ###def
###class

In [6]:
def run_hybrid_computing():
    print("=== Hybrid Computing Pipeline Start ===\n")

    # ---------------------------------------------------------
    # STEP 1: CPU 처리 (데이터 생성 및 전처리)
    # ---------------------------------------------------------
    print("[CPU] 1. Raw Data Generation")
    # 예: 5개의 센서 데이터라고 가정
    raw_data = np.random.rand(1, 5).astype(np.float32)
    print(f"   -> Input Data: {raw_data}")

    # ---------------------------------------------------------
    # STEP 2: GPU 처리 (AI 모델 추론)
    # ---------------------------------------------------------
    print("\n[GPU] 2. AI Model Inference")

    # GPU 사용 가능 여부 확인
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"   -> Processing on: {device}")

    # 모델과 데이터를 디바이스(GPU)로 이동
    model = SimpleClassifier().to(device)
    input_tensor = torch.from_numpy(raw_data).to(device)

    # 추론 실행 (Forward Pass)
    with torch.no_grad():
        ai_output = model(input_tensor)
    ###with

    # 결과 확인
    ai_result_value = ai_output.item() # GPU 메모리에서 값을 가져옴
    print(f"   -> AI Probability Output: {ai_result_value:.4f}")

    # ---------------------------------------------------------
    # STEP 3: CPU 재처리 (GPU 결과를 QPU 입력으로 변환)
    # ---------------------------------------------------------
    print("\n[CPU] 3. Data Re-processing (Encoding for Quantum)")
    # AI의 결과값(0~1)을 양자 회전 각도(0 ~ pi)로 매핑
    # 예: AI가 높은 확률을 주면 양자 상태를 |1> 쪽으로 더 많이 회전시킴
    theta = ai_result_value * math.pi
    print(f"   -> Converted Angle theta: {theta:.4f} radians")

    # ---------------------------------------------------------
    # STEP 4: QPU 처리 (양자 회로 실행)
    # ---------------------------------------------------------
    print("\n[QPU] 4. Quantum Circuit Execution")

    # 양자 회로 생성 (1 Qubit, 1 Classical Bit)
    qc = QuantumCircuit(1, 1)

    # 인코딩: Ry 게이트를 사용하여 theta만큼 상태 회전
    # 초기 상태 |0>에서 Ry(theta) -> cos(theta/2)|0> + sin(theta/2)|1>
    qc.ry(theta, 0)

    # 측정: 양자 상태를 고전 비트로 붕괴
    qc.measure(0, 0)

    # 시뮬레이터 설정 (실제 QPU 연결 시 IBM Quantum Provider 사용)
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)

    # 샷(Shots) 수: 양자 실험 반복 횟수
    shots = 1024
    job = simulator.run(compiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print(f"   -> Quantum Measurement Counts: {counts}")
    # ---------------------------------------------------------
    # STEP 5: CPU 최종 처리 (결과 해석)
    # ---------------------------------------------------------
    print("\n[CPU] 5. Final Decision Making")

    # 1이 측정된 비율 계산
    ones = counts.get('1', 0)
    probability_of_one = ones / shots

    print(f"   -> Measured Probability of state |1>: {probability_of_one:.4f}")

    if probability_of_one > 0.5:
        print("   => FINAL RESULT: POSITIVE (Based on AI + Quantum Confirmation)")
    else:
        print("   => FINAL RESULT: NEGATIVE")
    ###if

    print("\n=== Pipeline Complete ===")
###def

In [7]:
if __name__ == "__main__":
    run_hybrid_computing()
###if

=== Hybrid Computing Pipeline Start ===

[CPU] 1. Raw Data Generation
   -> Input Data: [[0.00050604 0.07971185 0.09519425 0.0960624  0.38305792]]

[GPU] 2. AI Model Inference
   -> Processing on: cuda
   -> AI Probability Output: 0.4234

[CPU] 3. Data Re-processing (Encoding for Quantum)
   -> Converted Angle theta: 1.3300 radians

[QPU] 4. Quantum Circuit Execution
   -> Quantum Measurement Counts: {'1': 382, '0': 642}

[CPU] 5. Final Decision Making
   -> Measured Probability of state |1>: 0.3730
   => FINAL RESULT: NEGATIVE

=== Pipeline Complete ===
